In [140]:
# PROJECT 4 - Wrangle Twitter data via API

## Table of Contents
* [Introduction](#intro)
* [Initial Brief](#1.1-initial-brief)
* [General Outline](#general_outline)
* [Import Libraries](#)
* [](#)

`Note: Fill at the end. Automate with python library/extension.`

# Introduction
Gather readily available data from an existing source on the web to allow first hand experience of wrangling data.<br>
It is a significant task as data will not always be provided and if it is: <br>
 - Best case: Spelling mistakes and/or equivalent,
 - Worst case: No schema/format, duplicates, incomplete and/or incorrect values recorded.

## Initial Brief
- User has provided archived twitter data for analysis
 - [ ] Twitter archive export in CSV
 - [ ] URL to Machine Learning image predictions
<br>
- Identify minimum:
 - [ ] 8 quality issues
 - [ ] 2 tidiness issues
<br>
- Out of scope:
 - [ ] Unique rating system
 - [ ] No gathering required past 01 Aug 2017

## General outline
- [ ] Read-in CSV data
- [ ] Access URL data (_over manually downloading file_)

In [141]:
## install modules via terminal
#pip install pandas # also downloads numpy
#pip install requests
#pip install tweepy

## Optional - provides TOC
#pip install jupyter_contrib_nbextensions

## Import Libraries

In [142]:
import pandas as pd
import numpy as np

import requests
import os

import msvcrt
import sys

from bs4 import BeautifulSoup

import tweepy

import json

## Defined Functions

- addFiles(filename)    `Created for the ability to scale`
- go_assess(df)         `Created to reiterate through assessment steps`

In [143]:
filelist = [] # declare
print('{} Files in list'.format(len(filelist)) ) # initial print

# Adds and tracks files
def add_files(*filename): # PARAMETER: <string>
    for file in filename:
        filelist.append(file)
        print('{} added to file list.'.format(file) )

    if len(filelist) > 1:
        print('{} files now in list.'.format( len(filelist)) )
    else:
        print('{} file now in list.'.format( len(filelist)) )
    return file

0 Files in list


In [144]:
def get_values(df, col, name): # 
    export = []
    value_cnt = col.value_counts()
    value = value_cnt.values
# test for duplicates, no duplicates should be equal to .series size
    if value.sum() > value.shape[0]: # there are duplicates
        txt_result = ('Duplicates found in column \'{}\', the max duplicate item repeats {} times.'.format(name, value.max()) ) # print results, return indexes
    else: # no duplicates
        txt_result = ('No duplicates found in column \'{}\'.'.format(name) )
        #print('{}: No duplicates found.'.format(col) )
    # pack variables into list
    export.append(value_cnt)
    export.append(txt_result)
    
    return export

In [145]:
#assessment = [] # create global
def go_assess(df):
    # empty every function call, to prevent list from accumulating over time
    results = [] #
    summary = [] #
    val_sum = [] # 
    assessment = []
    print('Dataframe contains the following columns:')
    print('{}\n'.format(df.columns) )

    for i, col in enumerate(df.columns):
        # copy into message
        print('Column {} - \'{}\' has been assessed. Assessment saved in results[{}] and summary[{}]'.format(i, col, i, i))
        
        # call and get results
        val_sum = get_values(df, df[col], col)

        # append results
        summary.append(val_sum[1])
        results.append(val_sum[0])

    assessment.append(summary)
    assessment.append(results)
    print('NOTE: To access variables, set a series name e.g below:\nseries[0][x] to access summary details.\nseries[1][x] to access the value_counts results.\nx represents column number')
    return assessment #

In [146]:
def trim_strings(df):
    for col in df:
        if df[col].dtype == 'object':
            startcount = df[col].str.len()
            df[col].str.strip()
            endcount = df[col].str.len()
            print

            if (startcount.sum() - endcount.sum()) > 0:
                print('Whitespaces were present in {}.'.format(col) )
            else:
                print('No whitespaces in {}.'.format(col) )

In [147]:
## BLANK

## Data Wrangling

## Iteration 1
Import data from a twitter user archive provided by the end-user

`Note: Add edit# upon addition of new issue.`

### Gathering 1
#### Initialize
Enter Known Input Info
Format: file name inside ''

In [148]:
# FILE 1 - TWITTER ARCHIVE DATA
folder = 'Incoming_Files/'
twitter_file = 'twitter-archive-enhanced-2.csv'
add_files(twitter_file)

twitter-archive-enhanced-2.csv added to file list.
1 file now in list.


'twitter-archive-enhanced-2.csv'

In [149]:
# FILE 2 - TWITTER ML IMAGE PREDICTIONS
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# assign to a response object
response = requests.get(url)

image_predictions = url.split('/')[-1] # extract file name

# with open, allows for the auto close file when complete
# split after last delimiter /, indicating file name
with open(os.path.join(folder, image_predictions), mode='wb') as file:
    # read file 
    file.write(response.content)
    print('{} has been saved in: "/{}"'.format(image_predictions, folder) )

# call function and add name to end of list
add_files(image_predictions)

image-predictions.tsv has been saved in: "/Incoming_Files/"
image-predictions.tsv added to file list.
2 files now in list.


'image-predictions.tsv'

In [150]:
run_script = str(input('Run script to Access Twitter API (Y/N)?'))
valid_input = ['n', 'N', 'y', 'Y']
yes_list = ['y','Y']
no_list = ['n','N']

In [151]:
# FILE 3 - TWITTER API JSON
# run python script, pass dataframe name (dataframe could not be passed)
while run_script not in valid_input:
    run_script = input('Wrong input. Run script to Access Twitter API (Y/N)?')

if run_script in yes_list:
    folderarg = folder.replace(' ', '_')
    print('Running. Will indicate when complete.\n')
    %run twitter-api.py $folderarg $filelist[0]
elif run_script in no_list:
    print('Script not running.')

Script not running.


In [152]:
## Twitter API data
API_cols = ['tweet_id', 'rt_count', 'fav_count']

# read in txt and convert to json
API_export = 'tweet_json.txt'
# call function and add name to end of list
add_files(API_export)

json_keys, json_id, json_fav_count, json_retweet_cnt = [], [],[],[]

with open(API_export) as txt_file:
    for line in txt_file:
        #print(line)
        json_obj = json.loads(line)
        #append to list then combine lists 
        json_keys.append(json_obj)
        json_id.append(json_obj['id_str'])
        json_fav_count.append(json_obj['favorite_count'])
        json_retweet_cnt.append(json_obj['retweet_count'])
        

tweet_json.txt added to file list.
3 files now in list.


#### Import into dataframes

In [153]:
 # create empty list
df_raw = []
file_extensions = []

# dataframe to contain original imports
for num, file in enumerate(filelist):
    ext = file.split('.')[-1]
    file_extensions.append(ext)
    # read extension type
    ## catch CSV, TSV, JSON, no Switch/Case in Python
    if ext == 'csv':
        df_raw.append(pd.read_csv(folder + file) )
    elif ext == 'tsv':
        df_raw.append(pd.read_csv(folder + file, sep='\t') )
    elif file == 'tweet_json.txt':
        df_raw.append(pd.DataFrame(zip(json_id, json_fav_count, json_retweet_cnt), columns=API_cols))
    else:
        print('filelist({}) - "{}", could not be read into a dataframe.'.format(num, filelist[num]) )

In [154]:
print(filelist)

['twitter-archive-enhanced-2.csv', 'image-predictions.tsv', 'tweet_json.txt']


In [155]:
df_raw[0].sample(3)  # visually assess file was read in correctly

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2137,670046952931721218,NaN,NaN,2015-11-27 01:10:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ben &amp; Carson. It's impossible for ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670046952...,11,10,Ben,None,None,None,None
1737,679530280114372609,NaN,NaN,2015-12-23 05:13:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Guys this really needs to stop. We've been ove...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/679530280...,7,10,a,None,None,None,None
1020,746757706116112384,NaN,NaN,2016-06-25 17:31:25 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Maddie. She gets some wicked air time....,NaN,NaN,NaN,https://vine.co/v/5BYq6hmrEI3,11,10,Maddie,None,None,None,None


In [156]:
df_twitter = df_raw[0].copy()

In [157]:
df_raw[1].sample(3)  # visually assess file was read in correctly

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
945,704480331685040129,https://pbs.twimg.com/media/CcbRIAgXIAQaKHQ.jpg,1,Samoyed,0.979206,True,Pomeranian,0.007185,True,Arctic_fox,0.006438,False
1274,750041628174217216,https://pbs.twimg.com/media/CmfssOtXYAAKa_Z.jpg,1,Labrador_retriever,0.252031,True,Maltese_dog,0.188090,True,golden_retriever,0.133017,True
1533,789986466051088384,https://pbs.twimg.com/media/CvaYgDOWgAEfjls.jpg,1,tub,0.479477,False,bathtub,0.325106,False,golden_retriever,0.078530,True


In [158]:
df_image_predictor = df_raw[1].copy() # create copy

In [159]:
df_raw[2].sample(3)  # visually assess file was read in correctly

,tweet_id,rt_count,fav_count
888,757597904299253760,0,294
321,832645525019123713,2883,521
1462,693109034023534592,1662,590


In [160]:
df_twitter_api = df_raw[2].copy()

In [161]:
# DO - search Incoming Files directory, files not in list to be added.

## Assessing data
### Assess 1 - Twitter Data Archive
#### Define:<br>


**Visual and programmatic summary**<br>
Exceptions:
1. ratings (numerator, denominator)

_Tidiness_<br>
1. Datatypes
1.1 Time stamp contains date and time, the timestamp can be split further
1.2 Columns 13-16 can be categorized into `Dog_Category`, values repeat the column name making it irrelevant

_Cleanliness_<br>
1 Missing information, Columns ordered by severity:<br>
1.1 Index 1-2 only has 78 non null values, a significant amount<br>
1.2 Index 6-8 contain 181 non null values<br>
1.3 Index 9 contains 2297 non null values<br>
2 Datatypes:<br>
2.1 float required for column 1-2 as the order is +17 providing no need for the precision of decimals


In [162]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [163]:
df_twitter.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [164]:
# call go_assess function
archive_assessed = go_assess(df_twitter)

Dataframe contains the following columns:
Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

Column 0 - 'tweet_id' has been assessed. Assessment saved in results[0] and summary[0]
Column 1 - 'in_reply_to_status_id' has been assessed. Assessment saved in results[1] and summary[1]
Column 2 - 'in_reply_to_user_id' has been assessed. Assessment saved in results[2] and summary[2]
Column 3 - 'timestamp' has been assessed. Assessment saved in results[3] and summary[3]
Column 4 - 'source' has been assessed. Assessment saved in results[4] and summary[4]
Column 5 - 'text' has been assessed. Assessment saved in results[5] and summary[5]
Column 6 - 'retweeted_status_id' has been assessed. Assessment saved in results[6] and 

### Column 0 - tweet_id

In [165]:
### Column 0 - 
archive_assessed[0][0], archive_assessed[1][0]

("No duplicates found in column 'tweet_id'.",
 749075273010798592    1
 741099773336379392    1
 798644042770751489    1
 825120256414846976    1
 769212283578875904    1
                      ..
 715360349751484417    1
 666817836334096384    1
 794926597468000259    1
 673705679337693185    1
 700151421916807169    1
 Name: tweet_id, Length: 2356, dtype: int64)

### Column 1 - in reply

In [166]:
### Column 1 - 
archive_assessed[0][1], archive_assessed[1][1]

("Duplicates found in column 'in_reply_to_status_id', the max duplicate item repeats 2 times.",
 6.671522e+17    2
 8.562860e+17    1
 8.131273e+17    1
 6.754971e+17    1
 6.827884e+17    1
                ..
 8.482121e+17    1
 6.715449e+17    1
 6.936422e+17    1
 6.849598e+17    1
 7.331095e+17    1
 Name: in_reply_to_status_id, Length: 77, dtype: int64)

In [167]:
df_twitter[df_twitter.in_reply_to_status_id.notna()]['in_reply_to_status_id'].sample(5)

565     7.331095e+17
611     7.971238e+17
1474    6.936422e+17
291     8.380855e+17
342     8.320875e+17
Name: in_reply_to_status_id, dtype: float64

In [168]:
### Column 2 - 
archive_assessed[0][2], archive_assessed[1][2]

("Duplicates found in column 'in_reply_to_user_id', the max duplicate item repeats 47 times.",
 4.196984e+09    47
 2.195506e+07     2
 7.305050e+17     1
 2.916630e+07     1
 3.105441e+09     1
 2.918590e+08     1
 2.792810e+08     1
 2.319108e+09     1
 1.806710e+08     1
 3.058208e+07     1
 2.625958e+07     1
 1.943518e+08     1
 3.589728e+08     1
 8.405479e+17     1
 2.894131e+09     1
 2.143566e+07     1
 2.281182e+09     1
 1.648776e+07     1
 4.717297e+09     1
 2.878549e+07     1
 1.582854e+09     1
 4.670367e+08     1
 4.738443e+07     1
 1.361572e+07     1
 1.584641e+07     1
 2.068372e+07     1
 1.637468e+07     1
 1.185634e+07     1
 1.198989e+09     1
 1.132119e+08     1
 7.759620e+07     1
 Name: in_reply_to_user_id, dtype: int64)

In [169]:
### Column 3 - 

In [170]:
archive_assessed[0][3], archive_assessed[1][3]

("No duplicates found in column 'timestamp'.",
 2016-07-28 01:00:57 +0000    1
 2016-10-18 16:11:17 +0000    1
 2017-06-10 00:35:19 +0000    1
 2016-01-08 03:50:03 +0000    1
 2016-07-04 15:00:45 +0000    1
                             ..
 2017-03-29 00:01:05 +0000    1
 2016-11-29 16:33:36 +0000    1
 2015-12-25 19:39:43 +0000    1
 2016-10-11 00:34:48 +0000    1
 2015-11-17 00:24:19 +0000    1
 Name: timestamp, Length: 2356, dtype: int64)

In [171]:
### Column 4 - 
archive_assessed[0][4], archive_assessed[1][4]

("Duplicates found in column 'source', the max duplicate item repeats 2221 times.",
 <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
 <a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
 <a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
 <a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
 Name: source, dtype: int64)

In [172]:
### Column 5 - 
archive_assessed[0][5], archive_assessed[1][5]

("No duplicates found in column 'text'.",
 Finally some constructive political change in this country. 11/10 https://t.co/mvQaETHVSb                                                                        1
 This is Bernie. He's taking his Halloween costume very seriously. Wants to be baked. 3/10 not a good idea Bernie smh https://t.co/1zBp1moFlX                     1
 This is Mia. She makes awful decisions. 8/10 https://t.co/G6TQVgTcZz                                                                                             1
 Here's a little more info on Dew, your favorite roaming doggo that went h*ckin viral. 13/10 \nhttps://t.co/1httNYrCeW https://t.co/KvaM8j3jhX                    1
 This is Terry. He's a Toasty Western Sriracha. Doubles as a table. Great for parties. 10/10 would highly recommend https://t.co/1ui7a1ZLTT                       1
                                                                                                                                          

In [173]:
### Column 6 - 
archive_assessed[0][6], archive_assessed[1][6]

("No duplicates found in column 'retweeted_status_id'.",
 7.757333e+17    1
 7.507196e+17    1
 6.742918e+17    1
 6.833919e+17    1
 8.269587e+17    1
                ..
 7.848260e+17    1
 7.806013e+17    1
 8.305833e+17    1
 7.047611e+17    1
 7.331095e+17    1
 Name: retweeted_status_id, Length: 181, dtype: int64)

In [174]:
### Column 7 - 
archive_assessed[0][7], archive_assessed[1][7]

("Duplicates found in column 'retweeted_status_user_id', the max duplicate item repeats 156 times.",
 4.196984e+09    156
 4.296832e+09      2
 5.870972e+07      1
 6.669901e+07      1
 4.119842e+07      1
 7.475543e+17      1
 7.832140e+05      1
 7.266347e+08      1
 4.871977e+08      1
 5.970642e+08      1
 4.466750e+07      1
 1.228326e+09      1
 7.992370e+07      1
 2.488557e+07      1
 7.874618e+17      1
 3.638908e+08      1
 5.128045e+08      1
 8.117408e+08      1
 1.732729e+09      1
 1.960740e+07      1
 1.547674e+08      1
 3.410211e+08      1
 7.124572e+17      1
 2.804798e+08      1
 1.950368e+08      1
 Name: retweeted_status_user_id, dtype: int64)

In [175]:
### Column 8 - 
archive_assessed[0][8], archive_assessed[1][8]

("No duplicates found in column 'retweeted_status_timestamp'.",
 2016-10-27 16:06:04 +0000    1
 2015-12-21 22:15:18 +0000    1
 2017-06-19 17:14:49 +0000    1
 2016-10-26 22:31:36 +0000    1
 2015-11-19 01:27:25 +0000    1
                             ..
 2016-09-27 02:53:48 +0000    1
 2016-06-25 17:31:25 +0000    1
 2016-04-09 02:47:55 +0000    1
 2016-10-19 15:37:03 +0000    1
 2015-11-24 03:51:38 +0000    1
 Name: retweeted_status_timestamp, Length: 181, dtype: int64)

In [176]:
### Column 9 - 
archive_assessed[0][9], archive_assessed[1][9]

("Duplicates found in column 'expanded_urls', the max duplicate item repeats 2 times.",
 https://twitter.com/dog_rates/status/839549326359670784/photo/1                                                                    2
 https://twitter.com/dog_rates/status/753375668877008896/photo/1                                                                    2
 https://twitter.com/dog_rates/status/762699858130116608/photo/1                                                                    2
 https://twitter.com/dog_rates/status/837820167694528512/photo/1,https://twitter.com/dog_rates/status/837820167694528512/photo/1    2
 https://twitter.com/dog_rates/status/698195409219559425/photo/1                                                                    2
                                                                                                                                   ..
 https://twitter.com/dog_rates/status/725729321944506368/photo/1                                            

In [177]:
### Column 10 - 
archive_assessed[0][10], archive_assessed[1][10]

("Duplicates found in column 'rating_numerator', the max duplicate item repeats 558 times.",
 12      558
 11      464
 10      461
 13      351
 9       158
 8       102
 7        55
 14       54
 5        37
 6        32
 3        19
 4        17
 1         9
 2         9
 420       2
 0         2
 15        2
 75        2
 80        1
 20        1
 24        1
 26        1
 44        1
 50        1
 60        1
 165       1
 84        1
 88        1
 144       1
 182       1
 143       1
 666       1
 960       1
 1776      1
 17        1
 27        1
 45        1
 99        1
 121       1
 204       1
 Name: rating_numerator, dtype: int64)

In [178]:
### Column 11 - 
archive_assessed[0][11], archive_assessed[1][11]

("Duplicates found in column 'rating_denominator', the max duplicate item repeats 2333 times.",
 10     2333
 11        3
 50        3
 80        2
 20        2
 2         1
 16        1
 40        1
 70        1
 15        1
 90        1
 110       1
 120       1
 130       1
 150       1
 170       1
 7         1
 0         1
 Name: rating_denominator, dtype: int64)

In [179]:
### Column 12 - name

In [180]:
archive_assessed[0][12], archive_assessed[1][12]

("Duplicates found in column 'name', the max duplicate item repeats 745 times.",
 None       745
 a           55
 Charlie     12
 Oliver      11
 Lucy        11
           ... 
 Kial         1
 Joey         1
 Mollie       1
 Jazz         1
 Arlen        1
 Name: name, Length: 957, dtype: int64)

## Assess 2 - Twitter Image Predictions

In [181]:
df_image_predictor.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
371,672980819271634944,https://pbs.twimg.com/media/CVbodBOUsAAb7jZ.jpg,1,car_mirror,0.232754,False,basset,0.219461,True,beagle,0.112397,True
1501,784431430411685888,https://pbs.twimg.com/media/CuLcNkCXgAEIwK2.jpg,1,miniature_poodle,0.744819,True,toy_poodle,0.243192,True,standard_poodle,0.010920,True
2005,877556246731214848,https://pbs.twimg.com/media/DC20wEcW0AAf59m.jpg,1,basset,0.995368,True,Welsh_springer_spaniel,0.001936,True,bathtub,0.000468,False


In [182]:
df_image_predictor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [183]:
df_image_predictor.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [184]:
img_assessed = go_assess(df_image_predictor)

Dataframe contains the following columns:
Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

Column 0 - 'tweet_id' has been assessed. Assessment saved in results[0] and summary[0]
Column 1 - 'jpg_url' has been assessed. Assessment saved in results[1] and summary[1]
Column 2 - 'img_num' has been assessed. Assessment saved in results[2] and summary[2]
Column 3 - 'p1' has been assessed. Assessment saved in results[3] and summary[3]
Column 4 - 'p1_conf' has been assessed. Assessment saved in results[4] and summary[4]
Column 5 - 'p1_dog' has been assessed. Assessment saved in results[5] and summary[5]
Column 6 - 'p2' has been assessed. Assessment saved in results[6] and summary[6]
Column 7 - 'p2_conf' has been assessed. Assessment saved in results[7] and summary[7]
Column 8 - 'p2_dog' has been assessed. Assessment saved in results[8] and summary[8]
Column 9 - 'p3' has been assessed. Assessm

In [185]:
### Column 0
img_assessed[0][0], img_assessed[1][0]

("No duplicates found in column 'tweet_id'.",
 685532292383666176    1
 826598365270007810    1
 692158366030913536    1
 714606013974974464    1
 715696743237730304    1
                      ..
 816829038950027264    1
 847971574464610304    1
 713175907180089344    1
 670338931251150849    1
 700151421916807169    1
 Name: tweet_id, Length: 2075, dtype: int64)

In [186]:
### Column 1
# search for files other then .jpg, use .split and sift through values
not_jpg = df_image_predictor[~df_image_predictor.jpg_url.str.contains('.jpg',)]
not_jpg.jpg_url

320    https://pbs.twimg.com/tweet_video_thumb/CVKtH-...
815    https://pbs.twimg.com/tweet_video_thumb/CZ0mhd...
Name: jpg_url, dtype: object

In [187]:
img_assessed[0][1], img_assessed[1][1]

("Duplicates found in column 'jpg_url', the max duplicate item repeats 2 times.",
 https://pbs.twimg.com/media/CpmyNumW8AAAJGj.jpg    2
 https://pbs.twimg.com/media/CV_cnjHWUAADc-c.jpg    2
 https://pbs.twimg.com/media/CdHwZd0VIAA4792.jpg    2
 https://pbs.twimg.com/media/Cwx99rpW8AMk_Ie.jpg    2
 https://pbs.twimg.com/media/Cs_DYr1XEAA54Pu.jpg    2
                                                   ..
 https://pbs.twimg.com/media/CxlPnoSUcAEXf1i.jpg    1
 https://pbs.twimg.com/media/CUf7UIaWUAEuKFr.jpg    1
 https://pbs.twimg.com/media/CU3NE8EWUAEVdPD.jpg    1
 https://pbs.twimg.com/media/CYLAWFMWMAEcRzb.jpg    1
 https://pbs.twimg.com/media/CVBokRSWsAADuXx.jpg    1
 Name: jpg_url, Length: 2009, dtype: int64)

In [188]:
### Column 2
img_assessed[0][2], img_assessed[1][2]

("Duplicates found in column 'img_num', the max duplicate item repeats 1780 times.",
 1    1780
 2     198
 3      66
 4      31
 Name: img_num, dtype: int64)

In [189]:
### Column 3
img_assessed[0][3], img_assessed[1][3]

("Duplicates found in column 'p1', the max duplicate item repeats 150 times.",
 golden_retriever      150
 Labrador_retriever    100
 Pembroke               89
 Chihuahua              83
 pug                    57
                      ... 
 bearskin                1
 scorpion                1
 china_cabinet           1
 coral_reef              1
 mortarboard             1
 Name: p1, Length: 378, dtype: int64)

In [190]:
is_ws = df_image_predictor[df_image_predictor.p1.str.contains(' ',)]
is_ws

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [191]:
mask = img_assessed[1][3] == 1
img_assessed[1][3][mask]

espresso         1
leaf_beetle      1
walking_stick    1
rapeseed         1
slug             1
                ..
bearskin         1
scorpion         1
china_cabinet    1
coral_reef       1
mortarboard      1
Name: p1, Length: 175, dtype: int64

In [192]:
### Column 4
img_assessed[0][4], img_assessed[1][4]

("Duplicates found in column 'p1_conf', the max duplicate item repeats 2 times.",
 0.366248    2
 0.713293    2
 0.375098    2
 0.636169    2
 0.611525    2
            ..
 0.713102    1
 0.765266    1
 0.491022    1
 0.905334    1
 1.000000    1
 Name: p1_conf, Length: 2006, dtype: int64)

In [193]:
### Column 5
img_assessed[0][5], img_assessed[1][5]

("Duplicates found in column 'p1_dog', the max duplicate item repeats 1532 times.",
 True     1532
 False     543
 Name: p1_dog, dtype: int64)

In [194]:
df_image_predictor.query('p1_dog == False').iloc[:, [0,1,3,5]]

,tweet_id,jpg_url,p1,p1_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,box_turtle,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,shopping_cart,False
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,hen,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,desktop_computer,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,three-toed_sloth,False
...,...,...,...,...
2026,882045870035918850,https://pbs.twimg.com/media/DD2oCl2WAAEI_4a.jpg,web_site,False
2046,886680336477933568,https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg,convertible,False
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,limousine,False
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,paper_towel,False


In [195]:
p1_false_results = df_image_predictor.query('p1_dog == False').iloc[:,:6]
p1_false_results

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False
...,...,...,...,...,...,...
2026,882045870035918850,https://pbs.twimg.com/media/DD2oCl2WAAEI_4a.jpg,1,web_site,0.949591,False
2046,886680336477933568,https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg,1,convertible,0.738995,False
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False


In [196]:
p1_false_results.groupby(['p1']).size()

p1
African_crocodile      1
African_grey           1
African_hunting_dog    1
American_black_bear    1
Angora                 2
                      ..
wombat                 4
wood_rabbit            3
wooden_spoon           1
wool                   2
zebra                  1
Length: 267, dtype: int64

In [197]:
### Column 6
img_assessed[0][6], img_assessed[1][6]

("Duplicates found in column 'p2', the max duplicate item repeats 104 times.",
 Labrador_retriever    104
 golden_retriever       92
 Cardigan               73
 Chihuahua              44
 Pomeranian             42
                      ... 
 waffle_iron             1
 drake                   1
 jigsaw_puzzle           1
 water_bottle            1
 bucket                  1
 Name: p2, Length: 405, dtype: int64)

In [198]:
### Column 7
img_assessed[0][7], img_assessed[1][7]

("Duplicates found in column 'p2_conf', the max duplicate item repeats 3 times.",
 0.069362    3
 0.027907    2
 0.193654    2
 0.271929    2
 0.003143    2
            ..
 0.138331    1
 0.254884    1
 0.090644    1
 0.219323    1
 0.016301    1
 Name: p2_conf, Length: 2004, dtype: int64)

In [199]:
### Column 8
img_assessed[0][8], img_assessed[1][8]

("Duplicates found in column 'p2_dog', the max duplicate item repeats 1553 times.",
 True     1553
 False     522
 Name: p2_dog, dtype: int64)

In [200]:
### Column 9
img_assessed[0][9], img_assessed[1][9]

("Duplicates found in column 'p3', the max duplicate item repeats 79 times.",
 Labrador_retriever    79
 Chihuahua             58
 golden_retriever      48
 Eskimo_dog            38
 kelpie                35
                       ..
 barbell                1
 go-kart                1
 croquet_ball           1
 ibex                   1
 rotisserie             1
 Name: p3, Length: 408, dtype: int64)

In [201]:
### Column 10
img_assessed[0][10], img_assessed[1][10]

("Duplicates found in column 'p3_conf', the max duplicate item repeats 2 times.",
 0.094759    2
 0.035711    2
 0.000428    2
 0.044660    2
 0.162084    2
            ..
 0.024007    1
 0.132820    1
 0.002099    1
 0.083643    1
 0.033835    1
 Name: p3_conf, Length: 2006, dtype: int64)

In [202]:
### Column 11
img_assessed[0][11], img_assessed[1][11]

("Duplicates found in column 'p3_dog', the max duplicate item repeats 1499 times.",
 True     1499
 False     576
 Name: p3_dog, dtype: int64)

## Assess 3 - Twitter API Raw Data

In [203]:
df_twitter_api.sample(3)

,tweet_id,rt_count,fav_count
1614,684188786104872960,3394,1127
1232,710588934686908417,4394,1795
1145,720785406564900865,3013,739


In [204]:
df_twitter_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   2331 non-null   object
 1   rt_count   2331 non-null   int64 
 2   fav_count  2331 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 54.8+ KB


In [205]:
df_twitter_api.describe()

,rt_count,fav_count
count,2331.000000,2331.000000
mean,7386.187473,2624.406692
std,11471.427460,4439.587366
min,0.000000,1.000000
25%,1283.000000,532.000000
50%,3207.000000,1225.000000
75%,9036.500000,3044.500000
max,152335.000000,75413.000000


In [206]:
#assess raw json api data
api_assessed = go_assess(df_twitter_api)

Dataframe contains the following columns:
Index(['tweet_id', 'rt_count', 'fav_count'], dtype='object')

Column 0 - 'tweet_id' has been assessed. Assessment saved in results[0] and summary[0]
Column 1 - 'rt_count' has been assessed. Assessment saved in results[1] and summary[1]
Column 2 - 'fav_count' has been assessed. Assessment saved in results[2] and summary[2]
NOTE: To access variables, set a series name e.g below:
series[0][x] to access summary details.
series[1][x] to access the value_counts results.
x represents column number


In [207]:
api_assessed = go_assess(df_twitter_api)

Dataframe contains the following columns:
Index(['tweet_id', 'rt_count', 'fav_count'], dtype='object')

Column 0 - 'tweet_id' has been assessed. Assessment saved in results[0] and summary[0]
Column 1 - 'rt_count' has been assessed. Assessment saved in results[1] and summary[1]
Column 2 - 'fav_count' has been assessed. Assessment saved in results[2] and summary[2]
NOTE: To access variables, set a series name e.g below:
series[0][x] to access summary details.
series[1][x] to access the value_counts results.
x represents column number


In [208]:
api_assessed[0][0], api_assessed[1][0]

("No duplicates found in column 'tweet_id'.",
 743895849529389061    1
 807059379405148160    1
 711363825979756544    1
 707021089608753152    1
 858860390427611136    1
                      ..
 687460506001633280    1
 822244816520155136    1
 765371061932261376    1
 813127251579564032    1
 667915453470232577    1
 Name: tweet_id, Length: 2331, dtype: int64)

In [209]:
api_assessed[0][1], api_assessed[1][1]

("Duplicates found in column 'rt_count', the max duplicate item repeats 163 times.",
 0        163
 355        4
 1974       4
 2562       3
 1253       3
         ... 
 6544       1
 401        1
 403        1
 405        1
 22527      1
 Name: rt_count, Length: 1954, dtype: int64)

In [210]:
api_assessed[0][2], api_assessed[1][2]

("Duplicates found in column 'fav_count', the max duplicate item repeats 5 times.",
 51       5
 445      5
 589      5
 41       5
 1161     4
         ..
 16705    1
 329      1
 4427     1
 339      1
 4096     1
 Name: fav_count, Length: 1676, dtype: int64)

In [211]:
### Assess Iteration 2

In [212]:
# Improve
df_clean = []
df_clean.append(df_twitter)
df_clean.append(df_image_predictor)
df_clean.append(df_twitter_api)

In [213]:
for df in df_clean:
    print(df.shape)

(2356, 17)
(2075, 12)
(2331, 3)


## Cleaning data
### Quality Issue 1:
#### Define:
col0: tweet_id data type change to string, all dataframes

#### Code:

In [214]:
q1 = 'tweet_id'

In [215]:
# Print previous data types 
df_image_predictor[q1].head(1)
for df in df_clean:
    print(df[q1].head(1))

0    892420643555336193
Name: tweet_id, dtype: int64
0    666020888022790149
Name: tweet_id, dtype: int64
0    892420643555336193
Name: tweet_id, dtype: object


In [216]:
# Convert to string
for df in df_clean:
    df[q1] = df[q1].astype(str)

#### Test

In [217]:
df_image_predictor[q1].head(1)
for i, df in enumerate(df_clean):
    print(df[q1].head(1))

0    892420643555336193
Name: tweet_id, dtype: object
0    666020888022790149
Name: tweet_id, dtype: object
0    892420643555336193
Name: tweet_id, dtype: object


### Quality issue 2:
#### Define:
col3: change timestamp datatype to datetime

#### Code:

In [218]:
df_clean[0].timestamp = pd.to_datetime(df_clean[0].timestamp)

#### Test:

In [219]:
df_clean[0].timestamp

0      2017-08-01 16:23:56+00:00
1      2017-08-01 00:17:27+00:00
2      2017-07-31 00:18:03+00:00
3      2017-07-30 15:58:51+00:00
4      2017-07-29 16:00:24+00:00
                  ...           
2351   2015-11-16 00:24:50+00:00
2352   2015-11-16 00:04:52+00:00
2353   2015-11-15 23:21:54+00:00
2354   2015-11-15 23:05:30+00:00
2355   2015-11-15 22:32:08+00:00
Name: timestamp, Length: 2356, dtype: datetime64[ns, UTC]

### Quality Issue 3:
#### Define:
col4: split string to remove html tag and extract content within

In [220]:
archive_assessed[1][4]

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

#### Code:
strip string prior to splitting

In [221]:
df_clean[0].iloc[:,4] = df_clean[0].iloc[:,4].str.strip()

In [222]:
# RESET COLUMN if coded incorrectly
df_clean[0].iloc[:,4] = df_raw[0].iloc[:,4]

In [223]:
#1 
df_clean[0].iloc[:,4] = df_clean[0].iloc[:,4].apply(lambda text: BeautifulSoup(text, 'html.parser').get_text())

In [224]:
df_clean[0].rename(columns={'source':'source_app'}, inplace=True)

#### Test:

In [225]:
df_clean[0].iloc[:,4].value_counts()

Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source_app, dtype: int64

### Quality Issue 4:
#### Define:
col1,2,6,7: change datatype from float to int
#### Code:

In [226]:
q4 = list(df_twitter.iloc[:0, [1,2,6,7]])
# Print previous data types 
for column in q4:
    print(df_twitter[column].head(0))

Series([], Name: in_reply_to_status_id, dtype: float64)
Series([], Name: in_reply_to_user_id, dtype: float64)
Series([], Name: retweeted_status_id, dtype: float64)
Series([], Name: retweeted_status_user_id, dtype: float64)


In [227]:
# Convert to string
for column in q4:
    df_twitter[column] = df_twitter[column].astype(str)

#### Test:

In [228]:
for column in q4:
    print(df_twitter[column].head(0))

Series([], Name: in_reply_to_status_id, dtype: object)
Series([], Name: in_reply_to_user_id, dtype: object)
Series([], Name: retweeted_status_id, dtype: object)
Series([], Name: retweeted_status_user_id, dtype: object)


### Quality Issue 5:
#### Define:
remove potential whitespaces across all string/objects, trim front and end as visual inspection appeared to show start of strings not inline when scrolling down.

In [229]:
df_clean[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       2356 non-null   object             
 2   in_reply_to_user_id         2356 non-null   object             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source_app                  2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         2356 non-null   object             
 7   retweeted_status_user_id    2356 non-null   object             
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

#### Code & Test:
`Improvement opportunity. Scan for object/string dtype and return if true to easily filter`

In [230]:
# call function
trim_strings(df_clean[0])

No whitespaces in tweet_id.
No whitespaces in in_reply_to_status_id.
No whitespaces in in_reply_to_user_id.
No whitespaces in source_app.
No whitespaces in text.
No whitespaces in retweeted_status_id.
No whitespaces in retweeted_status_user_id.
No whitespaces in retweeted_status_timestamp.
No whitespaces in expanded_urls.
No whitespaces in name.
No whitespaces in doggo.
No whitespaces in floofer.
No whitespaces in pupper.
No whitespaces in puppo.


In [231]:
trim_strings(df_clean[1])

No whitespaces in tweet_id.
No whitespaces in jpg_url.
No whitespaces in p1.
No whitespaces in p2.
No whitespaces in p3.


In [232]:
trim_strings(df_clean[2])

No whitespaces in tweet_id.


### Quality Issue 6:
#### Define:
df_image_predictor<br>
col3,6,9: change to lower case

In [233]:
q6 = list(df_image_predictor.iloc[:0, [3,6,9]])

In [234]:
#### Code:
q6 = list(df_image_predictor.iloc[:0, [3,6,9]])
# Print previous data types 
for column in q6:
    print(df_image_predictor[column].head(5))

0    Welsh_springer_spaniel
1                   redbone
2           German_shepherd
3       Rhodesian_ridgeback
4        miniature_pinscher
Name: p1, dtype: object
0                collie
1    miniature_pinscher
2              malinois
3               redbone
4            Rottweiler
Name: p2, dtype: object
0      Shetland_sheepdog
1    Rhodesian_ridgeback
2             bloodhound
3     miniature_pinscher
4               Doberman
Name: p3, dtype: object


In [235]:
# RESET COLUMN if coded incorrectly
df_clean[1].iloc[:, [3,6,9]] = df_raw[1].iloc[:, [3,6,9]]

In [236]:
for column in q6:
    df_image_predictor[column] = df_image_predictor[column].str.lower()

In [237]:
#### Test:
for column in q6:
    print(df_image_predictor[column].head(5))

0    welsh_springer_spaniel
1                   redbone
2           german_shepherd
3       rhodesian_ridgeback
4        miniature_pinscher
Name: p1, dtype: object
0                collie
1    miniature_pinscher
2              malinois
3               redbone
4            rottweiler
Name: p2, dtype: object
0      shetland_sheepdog
1    rhodesian_ridgeback
2             bloodhound
3     miniature_pinscher
4               doberman
Name: p3, dtype: object


### Quality Issue 7:
#### Define:
col1: rename from jpg_url to img_url

In [238]:
df_clean[1].iloc[:0, 1]

Series([], Name: jpg_url, dtype: object)

#### Code:

In [239]:
df_clean[1].rename(columns={'jpg_url':'img_url'}, inplace=True)

#### Test:

In [240]:
df_clean[1].iloc[:0, 1]

Series([], Name: img_url, dtype: object)

### Quality Issue 8:
#### Define:
col2: rename from img_num to conf_tweet_img

In [241]:
#### Code: - improve with def function
df_clean[1].iloc[:0, 2]

Series([], Name: img_num, dtype: int64)

In [242]:
### Quality Issue 8:
df_clean[1].rename(columns={'img_num':'conf_tweet_img'}, inplace=True)

In [243]:
#### Test:
df_clean[1].iloc[:0, 2]

Series([], Name: conf_tweet_img, dtype: int64)

### Quality Issue 9:
#### Define:
<br>check col12 to remove/replace incorrect names with None

#### Code:

In [244]:
names_list = df_clean[0].name.value_counts().index
names_list

Index(['None', 'a', 'Charlie', 'Oliver', 'Lucy', 'Cooper', 'Tucker', 'Lola',
       'Penny', 'Winston',
       ...
       'Tyrus', 'Jarod', 'Carll', 'Eleanor', 'Acro', 'Kial', 'Joey', 'Mollie',
       'Jazz', 'Arlen'],
      dtype='object', length=957)

In [245]:
# extract names - regex test
name_mask = df_clean[0].name.str.match('[^A-Z]')
name_mask.value_counts()

False    2247
True      109
Name: name, dtype: int64

In [246]:
df_clean[0].name[name_mask].value_counts()

a               55
the              8
an               7
very             5
quite            4
just             4
one              4
getting          2
not              2
mad              2
actually         2
space            1
all              1
officially       1
infuriating      1
such             1
incredibly       1
by               1
his              1
this             1
old              1
unacceptable     1
life             1
my               1
light            1
Name: name, dtype: int64

In [247]:
df_clean[0].name.where(~name_mask)

0        Phineas
1          Tilly
2         Archie
3          Darla
4       Franklin
          ...   
2351        None
2352         NaN
2353         NaN
2354         NaN
2355        None
Name: name, Length: 2356, dtype: object

In [248]:
df_clean[0].name = df_clean[0].name.where(~name_mask,None)

#### Test:

In [249]:
df_clean[0].name

0        Phineas
1          Tilly
2         Archie
3          Darla
4       Franklin
          ...   
2351        None
2352        None
2353        None
2354        None
2355        None
Name: name, Length: 2356, dtype: object

### Quality Issue 10:
#### Define:
<br>check numerator rating value is correct.

In [250]:
# preview of strings in text column
list(df_clean[0].text.sample(5) )

["Can't do better than this lol. 10/10 for the owner https://t.co/yrqGyMZhW6",
 'Doggo will persevere. 13/10\nhttps://t.co/yOVzAomJ6k',
 "This is Charles. He's camera shy. Tail longer than average. Doesn't look overwhelmingly fluffy. 6/10 would still pet https://t.co/rXvcElhoog",
 "Here's a pupper that's very hungry but too lazy to get up and eat. 12/10 (vid by @RealDavidCortes) https://t.co/lsVAMBq6ex",
 "We usually don't rate marshmallows but this one's having so much fun in the snow. 10/10 (vid by @kylejk24) https://t.co/NL2KwOioBh"]

#### Code:

In [251]:
# regex filter to extract 123.34/123 found visually and programmatically
df_clean[0]['rating'] = df_clean[0].text.str.extract(r'(\b\d{0,3}\.?\d{1,2}\/\d{2,3})', expand=True)

In [252]:
df_clean[0]['rating'].value_counts(dropna=False)

12/10       558
11/10       463
10/10       460
13/10       350
9/10        157
8/10        102
14/10        54
7/10         53
5/10         35
6/10         32
3/10         19
4/10         15
2/10          9
1/10          8
9/11          2
15/10         2
420/10        2
9.75/10       2
4/20          2
0/10          2
1776/10       1
143/130       1
7/11          1
13.5/10       1
007/10        1
20/16         1
11.26/10      1
960/00        1
50/50         1
144/120       1
165/150       1
182/10        1
84/70         1
88/80         1
99/90         1
204/170       1
11.27/10      1
9.5/10        1
11/15         1
666/10        1
.13/10        1
44/40         1
80/80         1
60/50         1
121/110       1
.10/10        1
17/10         1
45/50         1
NaN           1
Name: rating, dtype: int64

In [253]:
#remove .13 and .10 manually
# remove .13
x = df_clean[0].query('rating==".13/10"').rating.index[0]
df_clean[0]['rating'].iloc[x] = df_clean[0]['rating'].iloc[x].split('.')[1]
df_clean[0]['rating'].iloc[x]

'13/10'

In [254]:
# remove .10
x = df_clean[0].query('rating==".10/10"').rating.index[0]
df_clean[0]['rating'].iloc[x] = df_clean[0]['rating'].iloc[x].split('.')[1]
df_clean[0]['rating'].iloc[x]

'10/10'

In [255]:
df_clean[0].rating.value_counts(dropna=False), df_clean[0].rating.shape

(12/10       558
 11/10       463
 10/10       461
 13/10       351
 9/10        157
 8/10        102
 14/10        54
 7/10         53
 5/10         35
 6/10         32
 3/10         19
 4/10         15
 2/10          9
 1/10          8
 420/10        2
 9.75/10       2
 4/20          2
 9/11          2
 0/10          2
 15/10         2
 1776/10       1
 7/11          1
 13.5/10       1
 007/10        1
 50/50         1
 20/16         1
 960/00        1
 143/130       1
 144/120       1
 11.26/10      1
 165/150       1
 88/80         1
 99/90         1
 204/170       1
 11.27/10      1
 9.5/10        1
 11/15         1
 182/10        1
 666/10        1
 44/40         1
 80/80         1
 60/50         1
 121/110       1
 84/70         1
 17/10         1
 45/50         1
 NaN           1
 Name: rating, dtype: int64,
 (2356,))

In [256]:
# check for non regex matches
checknull = df_clean[0].rating.isnull()

In [257]:
list(df_clean[0][checknull].text), list(df_clean[0][checknull].rating)

(['Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx'],
 [nan])

In [258]:
df_clean[0].rating = df_clean[0].rating.where(~checknull,0/10)

In [259]:
df_clean[0].rating

0       13/10
1       13/10
2       12/10
3       13/10
4       12/10
        ...  
2351     5/10
2352     6/10
2353     9/10
2354     7/10
2355     8/10
Name: rating, Length: 2356, dtype: object

In [266]:
df_clean[0].rating.str.split('/', n=2, expand=True).value_counts()

0      1  
12     10     558
11     10     463
10     10     461
13     10     351
9      10     157
8      10     102
14     10      54
7      10      53
5      10      35
6      10      32
3      10      19
4      10      15
2      10       9
1      10       8
4      20       2
420    10       2
0      10       2
9      11       2
9.75   10       2
15     10       2
121    110      1
165    150      1
11.27  10       1
11.26  10       1
11     15       1
13.5   10       1
007    10       1
143    130      1
144    120      1
99     90       1
17     10       1
1776   10       1
182    10       1
20     16       1
960    00       1
44     40       1
45     50       1
50     50       1
60     50       1
666    10       1
7      11       1
80     80       1
84     70       1
88     80       1
9.5    10       1
204    170      1
dtype: int64

In [ ]:
df_clean[0].info()

In [ ]:
#### Test:
df_clean[0].rating.isnull().values.any()

### Tidiness Issue 1:
#### Define:
timestamp split into three columns, date, time, timezone

#### Code:


In [ ]:
df_twitter.timestamp.sample(5)

In [ ]:
df_twitter['date'] = df_twitter['timestamp'].dt.date 
df_twitter['time'] = df_twitter['timestamp'].dt.time 
df_twitter['timezone'] = df_twitter['timestamp'].astype(str).str[-6:]
df_twitter.drop(labels='timestamp', axis=1, inplace = True)

#### Test:


In [ ]:
df_clean[0].iloc[:,16:]

### Tidiness Issue 2:
#### Define:
categorize dog type into one column, and drop redundant columns.
### Quality Issue #:
#### Define:
change datatype into categorical

In [ ]:
df_twitter.iloc[:,11:16].sample(10)

In [ ]:
df_twitter['dog_type'] = df_twitter.text.str.extract('(doggo|floofer|pupper|puppo)', expand=False)
df_twitter['dog_type'] = df_twitter['dog_type'].astype('category')

In [ ]:
drop_cols = list(df_twitter.iloc[:1,12:16])
drop_cols

In [ ]:
df_twitter.drop(drop_cols, axis=1, inplace=True)

In [ ]:
#### Test:
df_twitter['dog_type'].value_counts(dropna=False)

In [ ]:
list(df_twitter.iloc[:0,:])

### Tidiness Issue 3:
#### Define:
merge dataframes to contain the relevant columns required for analysis ensuring each is relevant to the information it pertains.

#### Code:


In [ ]:
df_twitter.shape

In [ ]:
df_twitter_clean = pd.merge(df_clean[0], df_clean[2], on='tweet_id', how='inner')

#### Test:

In [ ]:
df_twitter_clean.shape

In [ ]:
df_twitter_clean.info()

In [ ]:
df_clea

## Save clean data